<a href="https://colab.research.google.com/github/StevenSong-sTs/ss24-capstone-team23-datallah-nkitts-steveso/blob/main/ss24_capstone_team23_datallah_nkitts_steveso/congress_req.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=0836460b1e07a06ff564f356104186c4c7b94a538b56e564ef79beb258579c0d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# import dependencies
import requests
import pandas as pd
import pprint
import json
import os
from tqdm import tqdm
import time
from pyspark.sql import SparkSession
from getpass import getpass

try: api_key
except: api_key = getpass('What is your Congress API Key? ')

What is your Congress API Key? ··········


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
shared_drive = 'drive/MyDrive/MADS Capstone Team 23/Data'
raw_path = f'{shared_drive}/raw/congress_data_files'
trns_path = f'{shared_drive}/processed/congress_data_files'

def req_data(folder_path, file_nm):
  '''
  Checks if file exists. If file exists, asks if user wants to reproduce data.
  '''

  if os.path.isfile(f'{folder_path}/{file_nm}'):
    rerun_ind = input(f'Would you like to overwrite {file_nm}? ')
    if rerun_ind.lower() == 'yes':
      print(f'Writing {file_nm}')
      return True
    else:
      return False
  else:
    print(f'Writing {file_nm}')
    return True


### Request Congressional Sessions

In [5]:
file_nm = 'congress_sessions'

if req_data(raw_path, file_nm):
  congress_req = f'https://api.congress.gov/v3/congress?api_key={api_key}&format=json&limit=250'
  congress_json = requests.get(congress_req)
  chambers = pd.json_normalize(congress_json.json()['congresses'],
                              record_path = 'sessions',
                              meta = ['name', 'startYear', 'endYear'])
  chambers['congress_num'] = chambers.name.str.extract(r'([0-9]{2,3})')
  chambers.to_csv(f'{raw_path}/{file_nm}', index = False)
  # create tab for congress years (inclusive)
  congress_yrs = chambers.groupby('congress_num', as_index = False).agg(
    start_yr = ('startYear', 'min'),
    final_yr = ('endYear', 'max'))
  congress_yrs = congress_yrs.astype(int)
  congress_yrs.to_csv(f'{raw_path}/congress_session_yrs', index = False)
else:
  chambers = pd.read_csv(f'{raw_path}/{file_nm}')
  congress_yrs = pd.read_csv(f'{raw_path}/congress_session_yrs')

Would you like to overwrite congress_sessions? no


In [6]:
chambers.sample(5)

,chamber,endDate,number,startDate,type,name,startYear,endYear,congress_num
347,Senate,1883-03-03,2.0,1882-12-04,R,47th Congress,1881,1883,47.0
414,House of Representatives,1862-07-17,2.0,1861-12-02,R,37th Congress,1861,1863,37.0
430,House of Representatives,1858-06-16,NaN,1858-06-15,S,35th Congress,1857,1859,35.0
52,House of Representatives,1997-11-13,1.0,1997-01-07,R,105th Congress,1997,1998,105.0
560,House of Representatives,1808-04-25,1.0,1807-10-26,R,10th Congress,1807,1809,10.0


In [7]:
congress_yrs.head()

,congress_num,start_yr,final_yr
0,10,1807,1809
1,100,1987,1988
2,101,1989,1990
3,102,1991,1992
4,103,1993,1994


### Request Members of each Session & Legislation

In [8]:
def paginate(api_key, endpoint, return_type, meta = None, record_path = None, req_format = 'json', req_limit = 250):
  '''
  Accepts a congress api endpoint and returns a fully paginated DataFrame

  api_key: api key as a string
  endpoint: congress api endpoing name (e.g. 'member')
  return_type: accepts either 'df' or 'list'
  meta: input list for json_normalize...only used if return_type == 'df'
  record_path: input list for json_normalize...only used if return_type == 'df'
  req_format: accepted values are 'xml' or 'json'
  req_limit: accepts an integer up to 250
  '''
  # grab initial link
  req = f'https://api.congress.gov/v3/{endpoint}?api_key={api_key}&format={req_format}&limit={req_limit}'
  json = requests.get(req)
  json_key_lst = list(json.json().keys())
  # check json object requires parsing
  endpoint_bool = '/' not in endpoint
  if endpoint_bool: json2 = json.json()[json_key_lst[0]]
  else: json2 = json.json()
  # check return type to pick procedure
  if return_type == 'df':
    # create empty df to fill
    df = pd.DataFrame()
    # paginate indefinitely until loop broken
    while True:
      # normalize and add to reps df
      temp_df = pd.json_normalize(json2,
                                  record_path = record_path,
                                  meta = meta,
                                  errors = 'ignore')
      df = pd.concat([df, temp_df], ignore_index = True)
      # if next link exists, continue loop, otherwise iterate through next chamber
      try:
        req = json2['pagination']['next'] + f'&api_key={api_key}'
        json = requests.get(req)
        if endpoint_bool: json2 = json.json()[json_key_lst[0]]
        else: json2 = json.json()
      except KeyError: break
    return df
  elif return_type == 'list':
    # create empty list to fill
    lst = []
    # paginate indefinitely until loop broken
    while True:
      # append to list
      lst.append(json2)
      # if next link exists, continue loop, otherwise iterate through next chamber
      try:
        req = json2['pagination']['next'] + f'&api_key={api_key}'
        json = requests.get(req)
        if endpoint_bool: json2 = json.json()[json_key_lst[0]]
        else: json2 = json.json()
      except KeyError: break
    return lst


In [9]:
file_nm = 'congress_members'

if req_data(raw_path, file_nm):
  # grab member IDs
  member_ids = paginate(api_key = api_key, meta = ['bioguideId'], endpoint = 'member', return_type = 'df')
  member_ids_ary = member_ids.bioguideId.unique()
  # iteratively develop JSON list
  terms_json_lst = []
  for member in member_ids_ary:
    endpoint = f'member/{member}'
    m_req = f'https://api.congress.gov/v3/{endpoint}?api_key={api_key}&format=json'
    m_json = requests.get(m_req)
    terms_json_lst.append(m_json.json()['member'])
  # save file
  with open(f'{raw_path}/congress_members_json', 'w') as output_file:
    json.dump(terms_json_lst, output_file, indent=2)
  # create df
  terms_df = pd.json_normalize(terms_json_lst,
                              record_path = ['terms'],
                              meta = ['bioguideId', 'currentMember', 'directOrderName', 'partyHistory',
                                      'sponsoredLegislation', 'cosponsoredLegislation', 'updateDate'],
                              errors = 'ignore')
  # restrict to 1999 congress and fwd
  terms_df = terms_df[terms_df.congress >= 106]
  # write df
  terms_df.to_csv(f'{raw_path}/{file_nm}', index = False)
else:
  terms_df = pd.read_csv(f'{raw_path}/{file_nm}')

Would you like to overwrite congress_members? no


In [10]:
terms_df.sample(5)

,chamber,congress,district,memberType,startYear,stateCode,stateName,endYear,bioguideId,currentMember,directOrderName,partyHistory,sponsoredLegislation,cosponsoredLegislation,updateDate
910,House of Representatives,116,9.0,Representative,2019,NJ,New Jersey,2021.0,P000096,True,"Bill Pascrell, Jr.","[{'partyAbbreviation': 'D', 'partyName': 'Demo...","{'count': 357, 'url': 'https://api.congress.go...","{'count': 4924, 'url': 'https://api.congress.g...",2024-06-08T18:40:19Z
1249,House of Representatives,112,6.0,Representative,2011,MO,Missouri,2013.0,G000546,True,Sam Graves,"[{'partyAbbreviation': 'R', 'partyName': 'Repu...","{'count': 253, 'url': 'https://api.congress.go...","{'count': 1944, 'url': 'https://api.congress.g...",2024-06-08T18:40:18Z
1047,House of Representatives,115,1.0,Representative,2017,MI,Michigan,2019.0,B001301,True,Jack Bergman,"[{'partyAbbreviation': 'R', 'partyName': 'Repu...","{'count': 76, 'url': 'https://api.congress.gov...","{'count': 785, 'url': 'https://api.congress.go...",2024-06-08T18:40:18Z
424,House of Representatives,112,28.0,Representative,2011,TX,Texas,2013.0,C001063,True,Henry Cuellar,"[{'partyAbbreviation': 'D', 'partyName': 'Demo...","{'count': 120, 'url': 'https://api.congress.go...","{'count': 2305, 'url': 'https://api.congress.g...",2024-06-08T18:40:21Z
2425,House of Representatives,107,NaN,Representative,2001,SD,South Dakota,2003.0,T000250,True,John Thune,"[{'partyAbbreviation': 'R', 'partyName': 'Repu...","{'count': 933, 'url': 'https://api.congress.go...","{'count': 2710, 'url': 'https://api.congress.g...",2024-03-10T12:42:24Z


### Grab Legislation

In [11]:
file_nm = 'spons_legis'

if req_data(raw_path, file_nm):
  # grab member IDs
  member_id_ary = terms_df.bioguideId.unique()
  # create empty file to update
  with open(f'{raw_path}/spons_legis_json', 'w') as f: pass
  # iterate over IDs and develop json file
  for member in tqdm(member_id_ary):
    endpoint = f'member/{member}/sponsored-legislation'
    try: temp_json = paginate(api_key, endpoint, 'list')
    except ValueError:
      print(f'Having trouble with {endpoint}.')
      print(f'Taking a break.')
      time.sleep(60 * 60)
      temp_json = paginate(api_key, endpoint, 'list')
    with open(f'{raw_path}/spons_legis_json', 'a') as f:
      for i in temp_json:
        f.write(json.dumps(i) + '\n')
  del temp_json
  # open file and create list
  spons_legis_json = []
  with open(f'{raw_path}/spons_legis_json', 'r') as f:
    for line in f:
      spons_legis_json.append(json.loads(line.strip()))
  # write df
  spons_legis = pd.json_normalize(spons_legis_json,
                                  record_path = ['sponsoredLegislation'],
                                  meta = [['request', 'bioguideId']],
                                  errors = 'ignore')
  spons_legis = spons_legis[['request.bioguideId', 'congress', 'amendmentNumber', 'number', 'introducedDate', 'type']]
  spons_legis.to_csv(f'{raw_path}/{file_nm}', index = False)
  del spons_legis_json
else:
  spons_legis = pd.read_csv(f'{raw_path}/{file_nm}')

Would you like to overwrite spons_legis? no


In [12]:
spons_legis.head()

,request.bioguideId,congress,amendmentNumber,number,introducedDate,type
0,g000579,118,NaN,8043.0,2024-04-17,HR
1,g000579,118,NaN,1051.0,2024-03-05,HRES
2,g000579,118,NaN,7229.0,2024-02-05,HR
3,g000579,118,NaN,7085.0,2024-01-25,HR
4,g000579,118,NaN,7037.0,2024-01-18,HR


In [13]:
file_nm = 'cospons_legis'

if req_data(raw_path, file_nm):
  # grab member IDs
  member_id_ary = terms_df.bioguideId.unique()
  # create empty file to update
  with open(f'{raw_path}/cospons_legis_json', 'w') as f: pass
  # iterate over IDs and develop json file
  for member in tqdm(member_id_ary):
    endpoint = f'member/{member}/cosponsored-legislation'
    try: temp_json = paginate(api_key, endpoint, 'list')
    except ValueError:
      print(f'Having trouble with {endpoint}.')
      print(f'Taking a break.')
      time.sleep(60 * 60)
      temp_json = paginate(api_key, endpoint, 'list')
    with open(f'{raw_path}/cospons_legis_json', 'a') as f:
      for i in temp_json:
        f.write(json.dumps(i) + '\n')
  del temp_json
  # open file and create list
  cospons_legis_json = []
  with open(f'{raw_path}/cospons_legis_json', 'r') as f:
    for line in f:
      cospons_legis_json.append(json.loads(line.strip()))
  # write df
  cospons_legis = pd.json_normalize(cospons_legis_json,
                                    record_path = ['cosponsoredLegislation'],
                                    meta = [['request', 'bioguideId']],
                                    errors = 'ignore')
  cospons_legis = cospons_legis[['request.bioguideId', 'congress', 'amendmentNumber', 'number', 'introducedDate', 'type']]
  cospons_legis.to_csv(f'{raw_path}/{file_nm}', index = False)
  del cospons_legis_json
else:
  cospons_legis = pd.read_csv(f'{raw_path}/{file_nm}')

Would you like to overwrite cospons_legis? no


In [14]:
cospons_legis.head()

,request.bioguideId,congress,amendmentNumber,number,introducedDate,type
0,g000579,118,NaN,8003.0,2024-04-15,HR
1,g000579,118,NaN,7942.0,2024-04-11,HR
2,g000579,118,NaN,3910.0,2024-03-19,HR
3,g000579,118,NaN,7373.0,2024-03-19,HR
4,g000579,118,NaN,117.0,2024-03-08,HJRES


### Create Legislation Features

In [15]:
del spons_legis, cospons_legis

In [16]:
# create a function to apply to partyHistory col to format
def partyHistory_format(json_str):
  return json.loads(json_str.replace("'", "\""))
# format the party history field into columns
terms_df['f_partyHistory'] = terms_df['partyHistory'].apply(partyHistory_format)
terms_df_exp = terms_df.explode('f_partyHistory').reset_index()
party_norm = pd.json_normalize(terms_df_exp.f_partyHistory)
f_terms_df = terms_df_exp.merge(party_norm, left_index = True, right_index = True)
f_terms_df.to_csv(f'{raw_path}/f_congress_members', index = False)
f_terms_df.head()

,index,chamber,congress,district,memberType,startYear_x,stateCode,stateName,endYear_x,bioguideId,...,directOrderName,partyHistory,sponsoredLegislation,cosponsoredLegislation,updateDate,f_partyHistory,partyAbbreviation,partyName,startYear_y,endYear_y
0,0,House of Representatives,118,4.0,Representative,2024,CO,Colorado,NaN,L000604,...,Greg Lopez,"[{'partyAbbreviation': 'R', 'partyName': 'Repu...",NaN,NaN,2024-07-13T18:40:13Z,"{'partyAbbreviation': 'R', 'partyName': 'Repub...",R,Republican,2024,NaN
1,1,House of Representatives,115,8.0,Representative,2017,WI,Wisconsin,2019.0,G000579,...,Mike Gallagher,"[{'partyAbbreviation': 'R', 'partyName': 'Repu...","{'count': 171, 'url': 'https://api.congress.go...","{'count': 885, 'url': 'https://api.congress.go...",2024-07-13T18:36:15Z,"{'partyAbbreviation': 'R', 'partyName': 'Repub...",R,Republican,2017,NaN
2,2,House of Representatives,116,8.0,Representative,2019,WI,Wisconsin,2021.0,G000579,...,Mike Gallagher,"[{'partyAbbreviation': 'R', 'partyName': 'Repu...","{'count': 171, 'url': 'https://api.congress.go...","{'count': 885, 'url': 'https://api.congress.go...",2024-07-13T18:36:15Z,"{'partyAbbreviation': 'R', 'partyName': 'Repub...",R,Republican,2017,NaN
3,3,House of Representatives,117,8.0,Representative,2021,WI,Wisconsin,2023.0,G000579,...,Mike Gallagher,"[{'partyAbbreviation': 'R', 'partyName': 'Repu...","{'count': 171, 'url': 'https://api.congress.go...","{'count': 885, 'url': 'https://api.congress.go...",2024-07-13T18:36:15Z,"{'partyAbbreviation': 'R', 'partyName': 'Repub...",R,Republican,2017,NaN
4,4,House of Representatives,118,8.0,Representative,2023,WI,Wisconsin,2024.0,G000579,...,Mike Gallagher,"[{'partyAbbreviation': 'R', 'partyName': 'Repu...","{'count': 171, 'url': 'https://api.congress.go...","{'count': 885, 'url': 'https://api.congress.go...",2024-07-13T18:36:15Z,"{'partyAbbreviation': 'R', 'partyName': 'Repub...",R,Republican,2017,NaN


In [17]:
spark = SparkSession.builder \
  .appName("Create Political Features") \
  .config("spark.executor.memory", "4g") \
  .config("spark.executor.cores", "4") \
  .getOrCreate()

# create sponsored legislation view
csv_file = f'{raw_path}/spons_legis'
df = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .csv(csv_file)
df.createOrReplaceTempView("spons_legis")

# create cosponsored legislation view
csv_file = f'{raw_path}/cospons_legis'
df2 = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .csv(csv_file)
df2.createOrReplaceTempView("cospons_legis")

# create members view
csv_file = f'{raw_path}/f_congress_members'
df3 = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .csv(csv_file)
df3.createOrReplaceTempView("f_congress_members")

# create congress yrs view
csv_file = f'{raw_path}/congress_session_yrs'
df4 = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .csv(csv_file)
df4.createOrReplaceTempView("congress_yrs")

In [21]:
query = f'''
  WITH mem AS (
    SELECT chamber,
      congress,
      district,
      stateCode,
      CASE
        WHEN stateCode IN ('NY') AND chamber = 'Senate' THEN 'New York'
        WHEN stateCode = 'NY' AND (district BETWEEN 1 AND 19) THEN 'New York'
        WHEN stateCode = 'CT' AND (district BETWEEN 3 AND 5) THEN 'New York'
        WHEN stateCode = 'NJ' AND (district BETWEEN 5 AND 12) THEN 'New York'
        WHEN stateCode IN ('IL') AND chamber = 'Senate' THEN 'Chicago'
        WHEN stateCode = 'IL' AND (district BETWEEN 1 AND 14) THEN 'Chicago'
        WHEN stateCode = 'IN' AND (district = 1) THEN 'Chicago'
        WHEN stateCode = 'WI' AND (district = 1) THEN 'Chicago'
        WHEN stateCode IN ('WA') AND chamber = 'Senate' THEN 'Seattle'
        WHEN stateCode = 'WA' AND district IN (1, 2, 6, 7, 8, 9, 10) THEN 'Seattle'
        WHEN stateCode IN ('TX') AND chamber = 'Senate' THEN 'Dallas'
        WHEN stateCode = 'TX' AND district IN (3,4,5,6,12,13,24,25,26,30,32,33) THEN 'Dallas'
        WHEN stateCode IN ('CA') AND chamber = 'Senate' THEN 'Los Angeles'
        WHEN stateCode = 'CA' AND district IN (23,26,27,29,30,28,32,31,33,
          34,37,36,35,39,41,48,49,40,47,46,45,44,43,42,38) THEN 'Los Angeles'
      END AS metro,
      memberType,
      startYear_x AS service_start_yr,
      COALESCE(endYear_x, 9999) AS service_end_yr,
      bioguideId,
      partyName,
      startYear_y AS party_start_yr,
      COALESCE(endYear_y, 9999) AS party_end_yr
    FROM f_congress_members
    WHERE (endYear_y IS NULL
        OR endYear_y < startYear_x))

  , spons AS (
    SELECT mem.*,
      sp.amendmentNumber,
      sp.number,
      sp.introducedDate AS spons_legis_intro_dt,
      YEAR(sp.introducedDate) spons_legis_intro_yr,
      MONTH(sp.introducedDate) spons_legis_intro_mnth
    FROM mem
    LEFT JOIN spons_legis sp
    ON mem.congress = sp.congress
      AND UPPER(mem.bioguideId) = UPPER(sp.`request.bioguideId`)
    WHERE YEAR(sp.introducedDate) BETWEEN party_start_yr AND party_end_yr)

  , cospons AS (
    SELECT mem.*,
      sp.amendmentNumber,
      sp.number,
      sp.introducedDate AS spons_legis_intro_dt,
      YEAR(sp.introducedDate) spons_legis_intro_yr,
      MONTH(sp.introducedDate) spons_legis_intro_mnth
    FROM mem
    LEFT JOIN cospons_legis sp
    ON mem.congress = sp.congress
      AND UPPER(mem.bioguideId) = UPPER(sp.`request.bioguideId`)
    WHERE YEAR(sp.introducedDate) BETWEEN party_start_yr AND party_end_yr)

  , spons_party_key AS (
    SELECT DISTINCT amendmentNumber,
      number,
      FIRST_VALUE(partyName) OVER (PARTITION BY amendmentNumber, number
                                   ORDER BY cnt DESC) AS legis_party
    FROM (
      SELECT amendmentNumber,
        number,
        partyName,
        COUNT(*) AS cnt
      FROM spons
      GROUP BY 1,2,3))

  , spons_party AS (
    SELECT spons.*, sp.legis_party
    FROM spons
    INNER JOIN spons_party_key sp
    ON ((spons.amendmentNumber = sp.amendmentNumber AND spons.number IS NULL)
      OR (spons.number = sp.number AND spons.amendmentNumber IS NULL)))

  , cospons_party AS (
    SELECT cospons.*, sp.legis_party
    FROM cospons
    INNER JOIN spons_party_key sp
    ON ((cospons.amendmentNumber = sp.amendmentNumber AND cospons.number IS NULL)
      OR (cospons.number = sp.number AND cospons.amendmentNumber IS NULL)))

  , mem_agg AS (
    SELECT spons_legis_intro_mnth,
      spons_legis_intro_yr,
      metro,
      COUNT(DISTINCT IF(chamber = 'Senate' AND partyName = 'Democratic', bioguideId, NULL)) AS dem_senate_cnt,
      COUNT(DISTINCT IF(chamber = 'Senate' AND partyName = 'Republican', bioguideId, NULL)) AS rep_senate_cnt,
      COUNT(DISTINCT IF(chamber != 'Senate' AND partyName = 'Democratic', bioguideId, NULL)) AS dem_house_cnt,
      COUNT(DISTINCT IF(chamber != 'Senate' AND partyName = 'Republican', bioguideId, NULL)) AS rep_house_cnt,
      COUNT(DISTINCT IF(chamber = 'Senate' AND partyName NOT IN ('Republican', 'Democratic'), bioguideId, NULL)) AS other_senate_cnt,
      COUNT(DISTINCT IF(chamber != 'Senate' AND partyName NOT IN ('Republican', 'Democratic'), bioguideId, NULL)) AS other_house_cnt
    FROM (
      SELECT * FROM spons
      UNION ALL
      SELECT * FROM cospons)
    WHERE metro IS NOT NULL
    GROUP BY 1,2,3)

  , spons_agg AS (
    SELECT spons_legis_intro_mnth,
      spons_legis_intro_yr,
      metro,
      COUNT(DISTINCT IF(partyName = 'Democratic', amendmentNumber, NULL)) +
        COUNT(DISTINCT IF(partyName = 'Democratic', number, NULL)) AS dem_spons_legis_cnt,
      COUNT(DISTINCT IF(partyName = 'Republican', amendmentNumber, NULL)) +
        COUNT(DISTINCT IF(partyName = 'Republican', number, NULL)) AS rep_spons_legis_cnt,
      COUNT(DISTINCT IF(partyName NOT IN ('Republican', 'Democratic'), amendmentNumber, NULL)) +
        COUNT(DISTINCT IF(partyName NOT IN ('Republican', 'Democratic'), number, NULL)) AS other_spons_legis_cnt
    FROM spons_party
    WHERE metro IS NOT NULL
    GROUP BY 1,2,3)

  , cospons_agg AS (
    SELECT spons_legis_intro_mnth,
      spons_legis_intro_yr,
      metro,
      COUNT(DISTINCT IF(partyName = 'Democratic', amendmentNumber, NULL)) +
        COUNT(DISTINCT IF(partyName = 'Democratic', number, NULL)) AS dem_cospons_legis_cnt,
      COUNT(DISTINCT IF(partyName = 'Republican', amendmentNumber, NULL)) +
        COUNT(DISTINCT IF(partyName = 'Republican', number, NULL)) AS rep_cospons_legis_cnt,
      COUNT(DISTINCT IF(partyName NOT IN ('Republican', 'Democratic'), amendmentNumber, NULL)) +
        COUNT(DISTINCT IF(partyName NOT IN ('Republican', 'Democratic'), number, NULL)) AS other_cospons_legis_cnt
    FROM cospons_party
    WHERE metro IS NOT NULL
    GROUP BY 1,2,3)

  , joined AS (
    SELECT
      COALESCE(m.spons_legis_intro_mnth, s.spons_legis_intro_mnth, c.spons_legis_intro_mnth) AS spons_legis_intro_mnth,
      COALESCE(m.spons_legis_intro_yr, s.spons_legis_intro_yr, c.spons_legis_intro_yr) AS spons_legis_intro_yr,
      COALESCE(m.metro, s.metro, c.metro) AS metro,
      m.dem_senate_cnt,
      m.rep_senate_cnt,
      m.dem_house_cnt,
      m.rep_house_cnt,
      m.other_senate_cnt,
      m.other_house_cnt,
      s.dem_spons_legis_cnt,
      s.rep_spons_legis_cnt,
      s.other_spons_legis_cnt,
      c.dem_cospons_legis_cnt,
      c.rep_cospons_legis_cnt,
      c.other_cospons_legis_cnt
    FROM mem_agg m
    FULL OUTER JOIN spons_agg s
    ON m.spons_legis_intro_mnth = s.spons_legis_intro_mnth
      AND m.spons_legis_intro_yr = s.spons_legis_intro_yr
      AND m.metro = s.metro
    FULL OUTER JOIN cospons_agg c
    ON m.spons_legis_intro_mnth = c.spons_legis_intro_mnth
      AND m.spons_legis_intro_yr = c.spons_legis_intro_yr
      AND m.metro = c.metro)

  SELECT *
  FROM joined

'''
# CREATE a view where the sponsored legislation is assigned to a party
# create a view where the cosponsored legislation is assigned to a party
# count number of unique reps from each chamber and party from spons table at each given month and metro (!)
# count number of democratic, republican, and other sponsored legislation for each month and metro (unique on bioguideid and legis number) (2)
# count number of democratic, republican, and other sponsored colegislation for each month and metro (unique on bioguideid and legis number) (3)
# join all three tables on month and metro

In [22]:
results = spark.sql(query)
# results.show()

In [23]:
results.toPandas().to_csv(f'{trns_path}/metro_politics.csv', header = True, index = False)